# Tutorial for Neural Networks

The purpose of this notebook is to create an implementation of a simple  artificial neural network for learning purposes. 

In [3]:
import numpy as np

In [4]:
X = np.array(([3,5],[5,1],[10,2]), dtype=float)
y = np.array(([75],[82],[93]), dtype=float)

In [5]:
# Normalize the data
X = X/np.amax(X, axis=0)
y = y/100

In [6]:
X

array([[ 0.3,  1. ],
       [ 0.5,  0.2],
       [ 1. ,  0.4]])

In [12]:
class NeuralNet(object):
    def __init__(self):
        self.num_layers = 3
        self.input_layer_size = 2
        self.output_later_size = 1
        self.hidden_layer_size  = 3
        
        self.w1 = np.random.randn(self.input_layer_size, self.hidden_layer_size)
        self.w2 = np.random.randn(self.hidden_layer_size, self.output_later_size)
        self.w = [self.w1, self.w2]
       
    def forward_propagation(self, X):
        self.z2 = np.dot(X, self.w1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.w2)
        self.y_hat = self.sigmoid(self.z3)
        
        return self.y_hat
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def sigmoid_derivative(self, z):
        return np.exp(-z)/(1+np.exp(-z)**2)
    
    def cost_function_derivative(self, X, y):
        self.y_hat = self.forward_propagation(X)
        
        delta3 = np.multiply(-(y-self.y_hat), self.sigmoid_derivative(self.z3))
        dj_dw2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.w2.T)*self.sigmoid_derivative(self.z2)
        dj_dw1 = np.dot(X.T, delta2)
        
        return dj_dw1, dj_dw2

There are a few important parts to building out a basic Artificial Neural Network. The first thing to do is define the concept of an activation/sigmoid function.

$$f(z)= \frac{1}{1+\exp(-x)}$$

This is the function applied to the inputs of all nodes in the network  and its result is passed to the next layer as an input. 

This activation function has a weight assigned to its input for each individual connection from one node to another. 

Implementation-wise, we use numpy arrays and matrix operations to make this function simpler and more time-efficient.


In [ ]:
def sigmoid(self, z):
    """Given an input, returns the result of the sigmoid function."""
    return 1/(1+np.exp(-z))

In [ ]:
def forward_propagation(self, X):
    self.z2 = np.dot(X, self.w1)
    self.a2 = self.sigmoid(self.z2)
    self.z3 = np.dot(self.a2, self.w2)
    self.y_hat = self.sigmoid(self.z3)

    return self.y_hat

def matrix_feed_forward_calc(self, X):
    for l in range(num_layers-1):
        if l == 0:
            node_in = x
        else:
            node_in = h
        z = self.w[l].dot(node_in) # + b[l] add bias later
        h = f(z)
    return h

In [8]:
def sigmoid(self, z):
    return 1/(1+np.exp(-z))

In [35]:
NN  = Neural_Net()

c1 = NN.cost_function_derivative(X, y)

In [36]:
 a,b = NN.cost_function_derivative(X,y)

In [37]:
a

array([[-0.06366729,  0.02766799,  0.25473514],
       [-0.04287048,  0.01929333,  0.18918441]])

In [38]:
b

array([[-0.2191875 ],
       [-0.32555071],
       [-0.19873838]])

$$c = \sqrt{a^2 + b^2}$$